# 🙂 **PyFeat**
<hr>
Original Github repo <a href="https://github.com/cosanlab/py-feat">here</a>. Adapted for <a href="https://mmla.gse.harvard.edu/">ez-mmla</a>, <a href="http://lit.gse.harvard.edu/">Harvard LITLab</a> (Schneider et al.)
<br><br>
<!-- Ideally, put a GIF or an image here that best portrays the demo or the model. -->
<center> <img width="600px" src="https://drive.google.com/uc?id=1mv-0KGy1ZWcdbpUrcJTvpSINL-tjyfZa">
</center>
<center>The results obtained from PyFeat </center>

> Py-Feat provides a comprehensive set of tools and models to easily detect facial expressions (Action Units, emotions, facial landmarks) from images and videos, preprocess & analyze facial expression data, and visualize facial expression data.

**Some notes:**
* This tool performs best when the face is clear and not too small. 
* For video detection, the model will be slow if you try to detect for every frame. 


## 🔧 **Setup**
<hr>
⏳ <b>Colab/Colab Pro</b>: 5 minutes


In [ ]:
#@title #### **Install and import dependencies**
#@markdown Install the PyFeat Python library and necessary dependencies.
from IPython.display import display, HTML, clear_output

completed_actions = []
current_action = ""

def start_step(action):
  global current_action
  current_action = action
  display(HTML(f"<b style='font-size: 15px'>{action} ⏳</b>"))
  display(HTML("<hr>"))

def finished():
  global current_action, completed_actions
  current_action = ""
  display(HTML(f"<b style='font-size: 15px; color: green;'>Done ✅ </b>"))
  completed_actions = []

def end_step():
  global current_action, completed_actions
  completed_actions.append(current_action)
  current_action = ""
  clear_output()
  for action in completed_actions:
    display(HTML(f"<b style='font-size: 15px'>{action} ✅ </b>"))
    display(HTML("<hr>"))

start_step("Install py-feat")
!pip install py-feat --q
end_step()

start_step("Install pytube")
!pip install pytube --q
end_step()

start_step("Install imageio")
!pip install imageio==2.4.1 --q
end_step()

start_step("Import necessary libraries")
from IPython.display import display, Javascript,HTML, Image
from google.colab.output import eval_js
from base64 import b64decode,b64encode
from google.colab import files
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
from scipy.io.wavfile import write
import os
from google.colab import drive
from pytube import YouTube 
from moviepy.editor import *
import cv2
from feat import Detector
end_step()

start_step("Load the PyFeat model")
#@title Load the Model (Detector)
from feat import Detector

detector = Detector(
    face_model="retinaface",
    landmark_model="mobilefacenet",
    au_model='jaanet',
    emotion_model="resmasknet",
    facepose_model="img2pose",
)
detector
end_step()

finished()

## 🕹 **Demo**
<hr>

> PyFeat allows for both image and video processing, so choose your input below!

In [ ]:
#@title #### **Image/video input**{ run: "auto" }
#@markdown Choose an option to input the audio or image for the demo and run this cell.
import ipywidgets as widgets
from google.colab import files
from IPython.display import Video, Image
from pytube import YouTube 
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode,b64encode
import cv2

input_file = None
image_or_video = "image"
input = "Demo picture" #@param ["Demo picture", "Take a picture", "Record video", "Upload file", "Upload from Google Drive"]

# The function takes in a dictionary of options and their corresponding functions
def dropdown_menu(options, on_finished):
    global input_file
    clear_output()
    # Run the associated function
    input_file = options[input]()
    clear_output()
    on_finished(input_file)

# This function runs whenever the action user chose finishes.
def on_finished(fn):
  global image_or_video
  if fn:
    display(HTML(f"<hr><b>Below is a preview of the file:</b> <code>{fn}</code><br><br>"))
    if fn[:-4] in ['.mp4', ".mov", ".avi"] or image_or_video == "video":
      image_or_video = "video"
      display(Video(fn, embed=True, width=400))
    else:
      image_or_video = "image"
      display(Image(fn, embed=True, width=400))

# Functions corresponding to the actions the user selected!
def demo():
  global image_or_video
  image_or_video = "image"

  youtube = YouTube('https://www.youtube.com/watch?v=XiRMa9wilVc&t=1s&ab_channel=LiSun')
  stream=youtube.streams.get_highest_resolution()
  videofile=stream.download()  
  vidcap = cv2.VideoCapture(videofile)
  success,image = vidcap.read()
  file="/content/image.jpg"
  cv2.imwrite(file, image)
  return file

def take_photo(filename='photo.jpg', quality=0.8):
  global image_or_video
  image_or_video = "image"
  
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

def record_video():
  global image_or_video
  image_or_video = "video"

  js=Javascript("""
    async function recordVideo() {
      const options = { mimeType: "video/webm; codecs=vp9" };
      const div = document.createElement('div');
      const capture = document.createElement('button');
      const stopCapture = document.createElement("button");
      
      capture.textContent = "Start Recording";
      capture.style.background = "orange";
      capture.style.color = "white";

      stopCapture.textContent = "Stop Recording";
      stopCapture.style.background = "red";
      stopCapture.style.color = "white";
      div.appendChild(capture);

      const video = document.createElement('video');
      const recordingVid = document.createElement("video");
      video.style.display = 'block';

      const stream = await navigator.mediaDevices.getUserMedia({audio:true, video: true});
    
      let recorder = new MediaRecorder(stream, options);
      document.body.appendChild(div);
      div.appendChild(video);

      video.srcObject = stream;
      video.muted = true;

      await video.play();

      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      await new Promise((resolve) => {
        capture.onclick = resolve;
      });
      recorder.start();
      capture.replaceWith(stopCapture);

      await new Promise((resolve) => stopCapture.onclick = resolve);
      recorder.stop();
      let recData = await new Promise((resolve) => recorder.ondataavailable = resolve);
      let arrBuff = await recData.data.arrayBuffer();
      
      // stop the stream and remove the video element
      stream.getVideoTracks()[0].stop();
      div.remove();

      let binaryString = "";
      let bytes = new Uint8Array(arrBuff);
      bytes.forEach((byte) => {
        binaryString += String.fromCharCode(byte);
      })
    return btoa(binaryString);
    }
  """)
  display(js)
  data=eval_js('recordVideo({})')
  binary=b64decode(data)
  with open("/content/recorded_video.mp4","wb") as video_file:
    video_file.write(binary)
  return "/content/recorded_video.mp4"
  
def drive_file_chooser(exts):
  # This function renders a file chooser which finds all the files of valid extensions
  # inside your Google Drive and allows the user to select the file
  files = []
  FILE_HTML = ""

  JS = """
  <style>
  .list-item {
    background: #eee;
    border-radius: 5px;
    padding: 5px;
    transition: all ease 0.1s;
    width: 50%;
    margin: 5px;
  }
  .list-item:hover {
    background: #ddd;
    transition: all ease 0.1s;
  }
  </style>
  <p> Below are the files obtained from your Google Drive. </p>
  <b> Choose a file from the list below. </b>
  <hr>
  <div> [!!!REPLACE!!!] </div>

  <script>
    var changestate;
    var selection = new Promise(resolve => {
      changestate = function(i) {
        var el = document.getElementById(i)
        el.style.backgroundColor = '#b0ceff';
        el.style.fontWeight = "900";
        resolve(i); 
      }
    });
  </script>
  """
  i = 0
  for root, directories, filenames in os.walk("/content/drive/My Drive"): 
    for filename in filenames:
      # change the extensions to the type of file we are looking for
      if filename.endswith(tuple(exts)):
          FILE_HTML += """<div id="{}" class="list-item" onclick="changestate('{}')">{}</div>""".format(i, i, filename)
          i += 1
          files.append(os.path.join(root,filename))
  display(HTML(JS.replace("[!!!REPLACE!!!]", FILE_HTML)))
  selection = eval_js("selection")
  return files[int(selection)]

def google_drive():
  from google.colab import drive
  display(HTML("<i>Please wait while we fetch files from your Google Drive...</i>"))
  drive.mount('/content/drive')
  clear_output()
  return drive_file_chooser(["png", "jpg", "mp4", "avi", "mov"])

def upload():
  try:
    uploaded = files.upload()
    fn = list(uploaded.keys())[0]
    return "/content/" + fn
  except:
    return None

dropdown_menu({
    "Demo picture": demo,
    "Take a picture": take_photo,
    "Record video": record_video,
    "Upload file": upload,
    "Upload from Google Drive": google_drive,
}, on_finished)


In [ ]:
#@title #### **Result output** { run: "auto" }
#@markdown Specify how you want to save your results and run this cell.
import time
result_output = None
output = "Don't save results" #@param ["Don't save results", "Download results", "Save to Google Drive"]

# The function takes in a dictionary of options and their corresponding functions
def dropdown_menu(options, on_finished):
    global result_output
    clear_output()
    # Run the associated function
    result_output = options[output]()
    clear_output()
    on_finished(result_output)

def on_finished(input):
  clear_output()
  global result_output
  if input:
    result_output = input
  if not input:
    return
  elif input == "download":
    display(HTML(f"<hr><b>Results will be downloaded when the demo is finished running.</b><br><br>"))
  else:
    display(HTML(f"<hr><b>Results will be stored to </b><code>{input}</code><b> when the demo is finished running.</b><br><br>"))

def drive_file_chooser():
  # This function renders a file chooser which finds all the files of valid extensions
  # inside your Google Drive and allows the user to select the file
  dirs = []
  FILE_HTML = ""

  JS = """
  <style>
  .list-item {
    background: #eee;
    border-radius: 5px;
    padding: 5px;
    transition: all ease 0.1s;
    width: 50%;
    margin: 5px;
  }
  .list-item:hover {
    background: #ddd;
    transition: all ease 0.1s;
  }
  </style>
  <p> Below are the folders obtained from your Google Drive. </p>
  <b> Choose a folder to store the model results. </b>
  <hr>
  <div> [!!!REPLACE!!!] </div>

  <script>
    var changestate;
    var selection = new Promise(resolve => {
      changestate = function(i) {
        var el = document.getElementById(i)
        el.style.backgroundColor = '#b0ceff';
        el.style.fontWeight = "900";
        resolve(i); 
      }
    });
  </script>
  """
  i = 0
  for root, directories, filenames in os.walk("/content/drive/My Drive"): 
    for directory in directories:
      # change the extensions to the type of file we are looking for
        FILE_HTML += """<div id="{}" class="list-item" onclick="changestate('{}')">{}</div>""".format(i, i, directory)
        i += 1
        dirs.append(os.path.join(root,directory))
  display(HTML(JS.replace("[!!!REPLACE!!!]", FILE_HTML)))
  selection = eval_js("selection")
  return dirs[int(selection)]

def google_drive():
  from google.colab import drive
  display(HTML("<i>Please wait while we fetch your Google Drive...</i>"))
  drive.mount('/content/drive')
  clear_output()
  return drive_file_chooser()

def nothing():
  return None

def download():
  return "download"

dropdown_menu({
    "Don't save results": nothing,
    "Save to Google Drive": google_drive,
    "Download results": download,
}, on_finished)


In [ ]:
#@title #### **Run the demo and save results**
#@markdown Run this cell to run the PyFeat model. <br> If you have uploaded a video, change the `skip_frames` parameter to change how the model processes the videos.

skip_frames = 30 #@param {type:"integer"}

from google.colab import files
if image_or_video == 'image':
  image_prediction = detector.detect_image(input_file)
  display(image_prediction)
else:
  video_prediction = detector.detect_video(input_file, skip_frames=skip_frames)
  display(video_prediction)

if image_or_video == 'image':
  figs = image_prediction.plot_detections(poses=True)[0]
else:
  axes = video_prediction.emotions.plot()
  video_prediction.plot_detections(faceboxes=False, add_titles=False)

# Handle output choice
if result_output == "download":
  if image_or_video == 'image':
      image_prediction.to_csv('pyfeat_result.csv')
      files.download('pyfeat_result.csv')
  else:
      video_prediction.to_csv('pyfeat_result.csv')
      files.download('pyfeat_result.csv')
elif result_output:
  if image_or_video == 'image':
      image_prediction.to_csv(os.path.join(result_output, 'pyfeat_result.csv'))
  else:
      video_prediction.to_csv(os.path.join(result_output, 'pyfeat_result.csv'))

display(HTML("<hr>"))
if result_output == "download":
  display(HTML(f"<b style='font-size: 15px'>Downloaded results</code> ✅"))
elif result_output:
  display(HTML(f"<b style='font-size: 15px'>Saved results to </b><code>{result_output}</code> ✅"))

